In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %%capture
!pip install sentence_transformers
!pip install kss
!pip install transformers

In [3]:
%cd /content/drive/Othercomputers/skt_노트북/ml

/content/drive/Othercomputers/skt_노트북/ml


In [13]:
##
from tqdm import tqdm
# from konlpy.tag import Mecab
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
import kss
import random
import transformers
from torch.utils.tensorboard import SummaryWriter

from load_dataset import *
# from preprocess import CustomTokenizer

from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader

In [5]:
## dataset
news = NaverSports()
docs, topics = news.load_data()

In [6]:
##
train_dataset = []
idxs = list(range(len(docs)))
for i in idxs:
    doc, topic = docs[i], topics[i]
    train_dataset += [InputExample(texts=[doc, topic], label=1.0)]
    new_idxs = idxs[:i] + idxs[i+1:]
    train_dataset += [InputExample(texts=[doc, topics[random.choice(new_idxs)]], label=0.0)]
    train_dataset += [InputExample(texts=[doc[random.choice(new_idxs)], topics], label=0.0)]

In [7]:
model = SentenceTransformer("jhgan/ko-sbert-nli")

In [15]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
train_loss = losses.CosineSimilarityLoss(model)
evaluator = evaluation.EmbeddingSimilarityEvaluator(docs, topics, scores=[1.0]*len(docs))
class LoggingLoss:
    def __init__(self, loss_fn=train_loss):
        self.loss_fn = loss_fn
    def __call__(self, logits, labels):
        loss = self.loss_fn(logits, labels)
        print(loss)

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, evaluator=evaluator, warmup_steps=100, 
          callback=LoggingLoss())
        #   callback=transformers.integrations.TensorBoardCallback(SummaryWriter()))

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/75 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


TypeError: ignored

In [15]:
model.save(path='./model')

In [16]:
mdoel = SentenceTransformer('./model')

In [ ]:
model.save_to_hub()